# Create ome-zarr from Single-Plane Multi-Field Acquisition

This example builds an ome-zarr (NGFF) high-content screening file from a `Molecular Devices ImageXpress` acquistion.

The example data is located in `../resources/Projection-Mix` and has the following form:
```
Storage hierarchy on disk for 2 wells with 2 fields and 3 channels:
    Projection-Mix --> {name}
    └── 2023-02-21 --> {date}
        └── 1334 --> {acquisition id}
            ├── Projection-Mix_E07_s1_w1E94C24BD-45E4-450A-9919-257C714278F7.tif
            ├── Projection-Mix_E07_s1_w1_thumb4BFD4018-E675-475E-B5AB-2E959E6B6DA1.tif
            ├── Projection-Mix_E07_s1_w2B14915F6-0679-4494-82D1-F80894B32A66.tif
            ├── Projection-Mix_E07_s1_w2_thumbE5A6D08C-AF44-4CA9-A6E8-7DC9BC9565F2.tif
            ├── Projection-Mix_E07_s1_w3BB87F860-FC67-4B3A-A740-A9EACF8A8F5F.tif
            ├── Projection-Mix_E07_s1_w3_thumbD739A948-8CA4-4559-910C-79D42CEC98A0.tif
            ├── Projection-Mix_E07_s2_w1DCFD1526-D063-4F8B-9E51-F1BD2EBD9F1A.tif
            ├── Projection-Mix_E07_s2_w1_thumbE4903920-91AA-405E-A39B-6EB284EA48E5.tif
            ├── Projection-Mix_E07_s2_w2607EE13F-AB5E-4E8C-BC4B-52E1118E7723.tif
            ├── Projection-Mix_E07_s2_w2_thumb15675A28-B45B-42E9-802C-833005E72657.tif
            ├── Projection-Mix_E07_s2_w3B0A47337-5945-4B26-9F5F-4EBA468CDBA9.tif
            ├── Projection-Mix_E07_s2_w3_thumbE3E91A92-3DD7-4EF5-BC32-8F5EF95B2CCA.tif
            ├── Projection-Mix_E08_s1_w117654C10-92F1-4DFD-98AA-6A01BBD77557.tif
            ├── Projection-Mix_E08_s1_w1_thumbD59B8F66-7743-4113-B6A2-8899FD53C3C8.tif
            ├── Projection-Mix_E08_s1_w281928711-999D-41F6-B88C-999513D4C092.tif
            ├── Projection-Mix_E08_s1_w2_thumb6D866DBC-F5F2-41D3-BEDD-28BEEB281FDD.tif
            ├── Projection-Mix_E08_s1_w3DD77D22D-07CB-4529-A1F5-DCC5473786FA.tif
            ├── Projection-Mix_E08_s1_w3_thumb615741CD-A218-4C30-BC79-E62A9D675DF5.tif
            ├── Projection-Mix_E08_s2_w1B38C01F5-0D36-4A29-9F5A-BE62B6F7F73F.tif
            ├── Projection-Mix_E08_s2_w1_thumbC7F53295-5DB8-4989-9E25-DD1068A2E266.tif
            ├── Projection-Mix_E08_s2_w266923EBB-9960-4952-8955-D1721D112EE2.tif
            ├── Projection-Mix_E08_s2_w2_thumb49A20B6B-1B86-47F1-B5FA-C22B47D2590D.tif
            ├── Projection-Mix_E08_s2_w3CCE83D85-0912-429E-9F18-716A085BB5BC.tif
            └── Projection-Mix_E08_s2_w3_thumb4D88636E-181E-4AF6-BC53-E7A435959C8F.tif

```

* Image data is stored in `{name}_{well}_{field}_w{channel}{md_id}.tif`.
* The `*_thumb*.tif` files are used by Molecular Devices as preview.

__Note:__ For this example we ignore the `ZStep_#` data.

In [1]:
from faim_hcs.io.MolecularDevicesImageXpress import parse_single_plane_multi_fields
from faim_hcs.Zarr import build_zarr_scaffold, write_cyx_image_to_well, PlateLayout, write_roi_table
from faim_hcs.MetaSeriesUtils import get_well_image_CYX, montage_grid_image_YX
from faim_hcs.UIntHistogram import UIntHistogram
import shutil
from tqdm.notebook import tqdm
from os.path import join, exists

In [2]:
acquisition_dir = '../resources/Projection-Mix/'
zarr_root = './zarr-files'

In [3]:
files = parse_single_plane_multi_fields(acquisition_dir)

In [4]:
files

,date,acq_id,name,well,field,channel,md_id,ext,path
0,2023-02-21,1334,Projection-Mix,E08,s2,w1,B38C01F5-0D36-4A29-9F5A-BE62B6F7F73F,.tif,../resources/Projection-Mix/2023-02-21/1334/Pr...
1,2023-02-21,1334,Projection-Mix,E08,s1,w2,81928711-999D-41F6-B88C-999513D4C092,.tif,../resources/Projection-Mix/2023-02-21/1334/Pr...
2,2023-02-21,1334,Projection-Mix,E08,s2,w3,CCE83D85-0912-429E-9F18-716A085BB5BC,.tif,../resources/Projection-Mix/2023-02-21/1334/Pr...
3,2023-02-21,1334,Projection-Mix,E07,s2,w3,B0A47337-5945-4B26-9F5F-4EBA468CDBA9,.tif,../resources/Projection-Mix/2023-02-21/1334/Pr...
4,2023-02-21,1334,Projection-Mix,E07,s1,w2,B14915F6-0679-4494-82D1-F80894B32A66,.tif,../resources/Projection-Mix/2023-02-21/1334/Pr...
5,2023-02-21,1334,Projection-Mix,E07,s2,w2,607EE13F-AB5E-4E8C-BC4B-52E1118E7723,.tif,../resources/Projection-Mix/2023-02-21/1334/Pr...
6,2023-02-21,1334,Projection-Mix,E08,s1,w3,DD77D22D-07CB-4529-A1F5-DCC5473786FA,.tif,../resources/Projection-Mix/2023-02-21/1334/Pr...
7,2023-02-21,1334,Projection-Mix,E08,s1,w1,17654C10-92F1-4DFD-98AA-6A01BBD77557,.tif,../resources/Projection-Mix/2023-02-21/1334/Pr...
8,2023-02-21,1334,Projection-Mix,E07,s1,w1,E94C24BD-45E4-450A-9919-257C714278F7,.tif,../resources/Projection-Mix/2023-02-21/1334/Pr...
9,2023-02-21,1334,Projection-Mix,E07,s2,w1,DCFD1526-D063-4F8B-9E51-F1BD2EBD9F1A,.tif,../resources/Projection-Mix/2023-02-21/1334/Pr...


In [8]:
if exists(join(zarr_root, "Single-Plane.zarr")):
    # Remove zarr if it already exists.
    shutil.rmtree(join(zarr_root, "Single-Plane.zarr"))

# Build empty zarr plate scaffold.
plate = build_zarr_scaffold(root_dir=zarr_root,
                            files=files,
                            name='Single-Plane',
                            layout=96,
                            order_name="example-order",
                            barcode="example-barcode")

In [9]:
channels = ['w1', 'w2', 'w3', 'w4']
# Add image data to wells
for well in tqdm(files['well'].unique()):
    well_files = files[files['well'] == well]
    
    img, hists, ch_metadata, metadata, roi_tables = get_well_image_CYX(
        well_files=well_files,
        channels=channels,
        assemble_fn=montage_grid_image_YX,
    )
    
    well_group = plate[well[0]][str(int(well[1:]))][0]
    write_cyx_image_to_well(img, hists, ch_metadata, metadata, well_group)
    
    # Write all ROI tables
    for roi_table in roi_tables:
        write_roi_table(roi_tables[roi_table], roi_table, well_group)

  0%|          | 0/2 [00:00<?, ?it/s]

# Inspect ome-zarr plate
The data can be opened with the [ome-zarr Napari plugin](https://www.napari-hub.org/plugins/napari-ome-zarr).